# Исследование надёжности заёмщиков банка

Проект для кредитного отдела банка. 

 **Задачи проекта:**
Определить, влияет ли *семейное положение* и *количество детей* клиента на *факт погашения кредита* в уложенный срок. Исходные данные от заказчика — статистика платёжеспособности клиентов.

Результаты исследования особенно нужны при построении **модели кредитного скоринга** — системы, оценивающей способность потенциального заёмщика вернуть взятый кредит.

## Датасет содержит следующие данные:

- `children` — количество детей в семье
    
- `days_employed` — общий трудовой стаж в днях
    
- `dob_years` — возраст клиента в годах
    
- `education` — уровень образования клиента
    
- `education_id` — идентификатор уровня образования
    
- `family_status` — семейное положение
    
- `family_status_id` — идентификатор семейного положения
    
- `gender` — пол клиента
    
- `income_type` — тип занятости
    
- `debt` — имел ли задолженность по возврату кредитов
    
- `total_income` — ежемесячный доход
    
- `purpose` — цель получения кредита

## Шаг 1. Знакомство данными и изение общей информации

In [3]:
import pandas as pd
data = pd.read_csv('data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


**Вывод**

<div class="alert alert-info">
<b>Из общей информации видно, что есть пропуски в столбцах 'days_employed' и 'total_income'. Кроме того, взглянув на перые 5 строк дафрейма видно, что в столбце 'days_employed' встечаются отрицательные значения.</b>
<br>
</div>


## Шаг 2. Предобработка данных

### Обработка пропусков

In [5]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [6]:
data_isna = data.loc[(data['days_employed'].isna() == True) | (data['total_income'].isna() == True)]

In [7]:
data_isna.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


In [8]:
data['days_employed'] = data['days_employed'].fillna(0)
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


In [9]:
type_income_median =  data.groupby('income_type', as_index=False)['total_income'].median().rename(columns={'total_income': 'median_income'})

In [10]:
type_income_median

,income_type,median_income
0,безработный,131339.751676
1,в декрете,53829.130729
2,госслужащий,150447.935283
3,компаньон,172357.950966
4,пенсионер,118514.486412
5,предприниматель,499163.144947
6,сотрудник,142594.396847
7,студент,98201.625314


In [11]:
data = data.merge(type_income_median, on='income_type', how='left')     
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,median_income
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,142594.396847
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,142594.396847
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,142594.396847
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,142594.396847
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,118514.486412
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем,172357.950966
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем,118514.486412
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость,142594.396847
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля,142594.396847


In [12]:
data.loc[data['total_income'].isna() == True, 'total_income'] = data['median_income']

In [13]:
data['children'].value_counts() #Есть категория граждан, у которых -1 ребенок (47 человек) и у которых 20 детей (76 человек). Что-то тут не так!

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

<div class="alert alert-info">

<b> </b> Я счиатаю, что когда указывались данные о детях, где поставили -1, имелось в виду значение 0, так как очень часто программа при отсутсвии значения возвращает -1, я думаю, это именно тот случай. Кроме того очень трудно опечататься, где клавиша "минус", а где 1, так что это точно не "тире", а именно минус. А там, где указано 20 детей, то имелось в виду два ребенка (так как в NumLock клавиша 2 и 0 расположены очень близко, можно легко промахнуться). Чтобы проверить эту гипотезу (для граждан, имеющих 20 детей), посмотрю возраст таких граждан и их ежемесячный доход, сравню его со средним доходом. Логично предположить: чтобы человек, имеющий 20 детей должен иметь доход явно выше среднего, чтобы обеспечитвать тех самых 20 детей. Проверю эту гипотезу.
</div>


In [14]:
data.groupby(data['children'])['dob_years'].mean() # средний возраст граждан в соответствии с количеством детей

children
-1     42.574468
 0     46.261432
 1     38.366127
 2     35.770316
 3     36.287879
 4     36.048780
 5     38.777778
 20    41.815789
Name: dob_years, dtype: float64

Видно, что средний возраст граждан с 20 детьми не сильно выбивается из средних значений по меньшему количеству детей

In [15]:
#введу переменную children_20, где сохраную данные о людях, имеющих 20 детей.
children_20 = data.loc[data['children'] == 20]
children_20.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,median_income
606,20,-880.221113,21,среднее,1,женат / замужем,0,M,компаньон,0,145334.865002,покупка жилья,172357.950966
720,20,-855.595512,44,среднее,1,женат / замужем,0,F,компаньон,0,112998.738649,покупка недвижимости,172357.950966
1074,20,-3310.411598,56,среднее,1,женат / замужем,0,F,сотрудник,1,229518.537004,получение образования,142594.396847
2510,20,-2714.161249,59,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474.835577,операции с коммерческой недвижимостью,142594.396847
2941,20,-2161.591519,0,среднее,1,женат / замужем,0,F,сотрудник,0,199739.941398,на покупку автомобиля,142594.396847


In [16]:
#средний доход и тип занятости для людей, имеющих 20 детей
type_income_20 = children_20.groupby('income_type', as_index=False)['total_income'].median()

In [17]:
type_income_20

,income_type,total_income
0,госслужащий,204226.869642
1,компаньон,171069.424797
2,пенсионер,127947.130141
3,сотрудник,142594.396847


In [18]:
type_income_median # средний доход по типу занятости для всех категорий граждан

,income_type,median_income
0,безработный,131339.751676
1,в декрете,53829.130729
2,госслужащий,150447.935283
3,компаньон,172357.950966
4,пенсионер,118514.486412
5,предприниматель,499163.144947
6,сотрудник,142594.396847
7,студент,98201.625314


In [19]:
#для наглядности сделаю единую таблицу, где объеденю данные
type_income_20.merge(type_income_median, on='income_type', how='inner')

,income_type,total_income,median_income
0,госслужащий,204226.869642,150447.935283
1,компаньон,171069.424797,172357.950966
2,пенсионер,127947.130141,118514.486412
3,сотрудник,142594.396847,142594.396847


Видно, что средний доход людей, имеющих 20 детей мало чем отличается от среднего дохода людей, имеющих намного меньше детей. 

<div class="alert alert-info">

<b></b> Гипотеза подтвердилась. Значит нужно изменить данные в столбце 'children'. Где было -1 заменяем на 0. Где было 20 заменяем на 2. 
</div>

In [20]:
data.loc[data['children'] == -1, 'children'] = 0

In [21]:
data.loc[data['children'] == 20, 'children'] = 2

In [22]:
data['children'].value_counts()

0    14196
1     4818
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [23]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
median_income       0
dtype: int64

**Вывод**

<div class="alert alert-info">
<b>
В столбцах `days_employed` и `total_income` одинаковое количество пропущенных значений (2174) в соответсвующих строках. Пропуски могут быть связаны с тем, что это либо неработающий пенсионер, либо, возможно, люди не захотели указывать свой трудовой стаж и ежемесячный доход, либо они работают неофициально. Это вызывает вопрос: как без данных о общем трудовом стаже в днях и ежемесячном доходе банк одобрил кредит?  
    
Заполнила пропуски столбца `days_employed` (общий трудовой стаж в днях) значением 0 (так как считаю, что в дальнейших расчетах это никак не повлияет), используя fillna(0).
    
Для того, чтобы заполнить пропуски в столбце `total_income` (ежемесячный доход), т.к. это важный показатель, для начала я сгруппировала датафрейм по столбцу `income_type` (методом  df.groupby('column') ), и по сгруппированным данным рассчитала медианные значения (методом data['column'].median()), сохраняя в переменную type_income_median.     
    
Используя merge(), соединила два дарафрейма. Затем заменила пропуски в столбце `total_income` медианным значением для конкретной категории. Еще раз убедилась, что в датафрейме больше нет пропусков. 

В столбце `children` обнаружила граждан, у которых -1 ребенок (47 человек) и у которых 20 детей (76 человек). Выдвинула гипотезу, что  там где поставили -1, имелось в виду значение 0, так как очень часто программа при отсутсвии значения возвращает -1, я думаю, это именно тот случай. Кроме того очень трудно опечататься, где клавиша "минус", а где 1, так что это точно не "тире", а именно минус. А там, где указано 20 детей, то имелось в виду два ребенка (так как в NumLock клавиша 2 и 0 расположены очень близко, можно легко промахнуться). Чтобы проверить эту гипотезу (для граждан, имеющих 20 детей), посмотрю возраст таких граждан и их ежемесячный доход, сравню его со средним доходом. Логично предположить: чтобы человек, имеющий 20 детей должен иметь доход явно выше среднего, чтобы обеспечитвать тех самых 20 детей. Проведя нужные подсчеты, подтвердила гипотезу: средний возраст и ежемесячный доход граждан с 20 детьми не сильно выбивается из средних значений, что подтверждает гипотезу.

Заменила данные в столбце `children`. Где было -1 заменила на 0. Где было 20 заменила на 2. 
    
Убедилась в отсутствии пропусков в столбцах датасета.
</b>
<br>
</div>

### Замена типа данных

In [24]:
data[['days_employed', 'total_income']] = data[['days_employed', 'total_income']].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
median_income       21525 non-null float64
dtypes: float64(1), int64(7), object(5)
memory usage: 2.9+ MB


**Вывод**

<div class="alert alert-info">
<b>Выполнила замену вещественного типа данных (в столбцах 'days_employed' и 'total_income', где хранятся данные типа float) на целочисленный c помощью astype('int'), потому что он позволяет изменить тип данных. Убедилась, что все применилось - info().</b>
<br>
</div>

### Обработка дубликатов

In [25]:
data.duplicated().sum()

54

In [26]:
data = data.drop_duplicates().reset_index(drop=True)

In [27]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,median_income
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,142594.396847
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,142594.396847
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,142594.396847
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,142594.396847
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,118514.486412


In [28]:
data['education'].value_counts()  

среднее                13705
высшее                  4710
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   273
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [29]:
data['family_status'].value_counts() 

женат / замужем          12344
гражданский брак          4163
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

In [30]:
data['gender'].value_counts() 

F      14189
M       7281
XNA        1
Name: gender, dtype: int64

In [31]:
data['income_type'].value_counts()  

сотрудник          11091
компаньон           5080
пенсионер           3837
госслужащий         1457
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

In [32]:
data['purpose'].value_counts() 

свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
покупка жилья для сдачи                   652
операции с жильем                         652
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [33]:
data['education'] = data['education'].str.lower()

In [34]:
data['education'].value_counts() 

среднее                15188
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [35]:
data.duplicated().sum()

17

In [36]:
data = data.drop_duplicates().reset_index(drop=True)

In [37]:
data.duplicated().sum()

0

 **Вывод**

<div class="alert alert-info">
<b>  
    
1. Выявила явные дубликаты и их количество, используя duplicated().sum()
    
2. Избавилась от явных дубликатов, используя drop_duplicates().reset_index(drop=True).
    
3. Поиск неявных дубликатов осуществяю с помощью метода value_counts(). Применяю этот способ ко всем столбцам, имеющим тип object. Вижу, что в столбце `education` строки написаны в разном регистре. Кроме того, в столбце `purpose` видны слова, записанные в разных формах, в следущием задании приведем слова к их словарной форме (лемме). Еще, в столбце 'gender' выявилось непонятное значение XNA 1 раз. Но, на дальнейшие расчеты это мало повляет. Пока не знаю, стоит ли с этим что-то делать? Предполагаю, что это `XNA`, скорее всего в английской традиции ‘seX Not Approved’ - то есть данные о гендерной принадлежности не предоставлены. 
    
4. Приведем строки столбца `education` к одному регистру с помощью функции и использоания метода str.lower().
    
5. Снова выявила явные дубликаты и их количество, используя duplicated().sum(). Удалила их, используя drop_duplicates().reset_index(drop=True). Убедилась, что избавилась от дубликатов.
    
    Причиной появления дубликатов считаю человеческий фактор, данные вводились невнимательно, повторяясь, с ошибками и с использованием разного регистра.
    
    </b>
<br>
</div>

### Лемматизация

In [38]:
from pymystem3 import Mystem
m = Mystem() 
purpose_list = data['purpose'].unique()
purpose_list

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [39]:
lemmas = []
for row in purpose_list:
    lemma = m.lemmatize(row)
    lemmas.append(lemma)
lemmas

[['покупка', ' ', 'жилье', '\n'],
 ['приобретение', ' ', 'автомобиль', '\n'],
 ['дополнительный', ' ', 'образование', '\n'],
 ['сыграть', ' ', 'свадьба', '\n'],
 ['операция', ' ', 'с', ' ', 'жилье', '\n'],
 ['образование', '\n'],
 ['на', ' ', 'проведение', ' ', 'свадьба', '\n'],
 ['покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', '\n'],
 ['покупка', ' ', 'недвижимость', '\n'],
 ['покупка', ' ', 'коммерческий', ' ', 'недвижимость', '\n'],
 ['покупка', ' ', 'жилой', ' ', 'недвижимость', '\n'],
 ['строительство', ' ', 'собственный', ' ', 'недвижимость', '\n'],
 ['недвижимость', '\n'],
 ['строительство', ' ', 'недвижимость', '\n'],
 ['на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', '\n'],
 ['на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', '\n'],
 ['операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', '\n'],
 ['строительство', ' ', 'жилой', ' ', 'недвижимость', '\n'],
 ['жилье', '\n'],
 ['операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', '\n'],
 ['автомобиль'

In [40]:
def purpose_change(purpose):
    lemmas_row = m.lemmatize(purpose)    
    for value in lemmas_row:
        if 'авто' in value:
            return 'автомобиль'
        if 'недвиж' in value or 'жил' in value:
            return  'недвижимость'
        if 'свад' in value:
            return  'свадьба'
        if 'образов' in value:
            return 'образование'
        
data['purpose']= data['purpose'].apply(purpose_change)        
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,median_income
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость,142594.396847
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль,142594.396847
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость,142594.396847
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование,142594.396847
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба,118514.486412


In [41]:
data.duplicated().sum()

335

In [42]:
data = data.drop_duplicates().reset_index(drop=True)

**Вывод**

<div class="alert alert-info">
<b> 
    
1. Загружаю библиотеку и сохраняю в список `purpose_list` уникальные значения столбца `purpose` с помощью метода unique().
    
2. Создаю пустой список lemmas, далее с помощью цикла каждую строчку в purpose_list лемматизирую, получаю список лемм и добавляю значения в пустой список lemmas, проверяю.
    
3. Создаю функцию purpose_change, которая разбивает на категории по леммам: "Жилье/недвижимость", "автомобиль", "образование", "свадьба". И заменила столбец `purpose`, в котором теперь хранится результат разбивки целей получения кредита по 4 категориям.
    
4. Проверила датафрейм на наличие дубликатов. Появилось 334 дубликата. Удалила их с помощью drop_duplicates().  
</b>
<br>
</div>

### Категоризация данных

Категоризирую данные по наличию детей (есть дети / нет детей):

In [43]:
def children_group(children):
    if children == 0:
        return 'Нет детей'
    if children >= 1:
        return 'Есть дети'
data['children_group'] = data['children'].apply(children_group)
data['children_group'].value_counts()

Нет детей    13875
Есть дети     7244
Name: children_group, dtype: int64

Ищу уникальные значения столца 'family_status'.

In [44]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

<div class="alert alert-info">
<b> 
Решила не категоризировать данные по семейному положению. Лишь вывела уникальные значения столбца 'family_status', чтобы посмотреть на категории.
   
</b>
<br>
</div>

Катекоризирую данные по уровню ежемесячного дохода, используя метод qcut(). 

In [45]:
data['total_income_group'] = pd.qcut(data['total_income'], 4)

In [46]:
data['total_income_group'].value_counts()

(196975.5, 2265604.0]    5280
(106808.0, 142594.0]     5280
(20666.999, 106808.0]    5280
(142594.0, 196975.5]     5279
Name: total_income_group, dtype: int64

Получились категории c интервалами: 
(20666.999 - 106808.0), 
(106808.0 - 142594.0), 
(142594.0 - 196975.5), 
(196975.5 - 2265604.0)

Итого 4 категории:

- меньше 106_808
- от 106_808 до 142_594
- от 142_594 до 196_975
- от 196_975

Категоризирую данные по возрасту. Выделяю категории 3 категории: до 18 лет - несовершеннолетние, от 18 до 60 лет и старше 60 лет.

In [47]:
def clients_age(age):
    if age < 18:
        return 'Несовершеннолетние'
    if 18 <= age <= 60:
        return 'Возраст до 60 лет'
    else:
        return 'Возраст более 60 лет'
    
data['clients_age_group'] = data['dob_years'].apply(clients_age)
data['clients_age_group'].value_counts()

Возраст до 60 лет       18940
Возраст более 60 лет     2079
Несовершеннолетние        100
Name: clients_age_group, dtype: int64

**Вывод**

<div class="alert alert-info">
<b> 
    
1. Большая часть заемщиков не имеет детей.
    
2. Категоризировав данные о ежемесячном доходе, чтобы сделать категории примерно равными, прибегла к методу qcut(), получила 4 категории.

3. Основная масса заемщиков - люди от 18-60 лет, значительно меньшая доля людей страше 60 лет. Нашла, что есть 100 несовершеннолетних, ниже провожу исследование этой категории, где прихожу к выводу, что это просто отсутствие данных. Решаю заполнить эти значения медианным значением.
</b>
<br>
</div>

In [48]:
data.groupby(data.loc[data['clients_age_group'] == 'Несовершеннолетние', 'clients_age_group'])['dob_years'].value_counts()

clients_age_group   dob_years
Несовершеннолетние  0            100
Name: dob_years, dtype: int64

In [49]:
median_age = data['dob_years'].median()
median_age

42.0

In [50]:
data.loc[data['dob_years'] == 0, 'dob_years'] = median_age

In [51]:
data.loc[data['dob_years'] == 0, 'dob_years'].sum()

0.0

In [52]:
data['clients_age_group'] = data['dob_years'].apply(clients_age)
data['clients_age_group'].value_counts()

Возраст до 60 лет       19040
Возраст более 60 лет     2079
Name: clients_age_group, dtype: int64

<div class="alert alert-info">
<b> 
Посмотрела возраст несовершеннолетних. 
</b> У этой категории возраст = 0. Это говорит о том, что данные о возрасте попросту не заполнены.
    Решаю заполнить эти данные медианным значением возраста (введя переменную median_age, в которой к столбцу 'dob_years' применила метод median()). Медианное значение возраста = 42 года. Заполню этим значением столбец 'dop_years', где значение = 0. Удедилась, что нуля больще нет. Убедилась, что нет совершеннолетних. Теперь у нас 2 категории по возрасту и видно, что преобладает катерогия совершеннолетних граждан до 60 лет (что логично). Вывод: Банк не выдовал кредит несовершеннолетним.
    
<br>
</div>

## Шаг 3. Ответьте на вопросы

<div class="alert alert-info">
<b> 
Для ответа на вопросы 3 шага применила сводные таблицы (метод `pivot_table`).
</b>    
<br>
</div>

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Расчитываю долю неплатильщиков для 2 категорий (есть дети/нет детей)

In [53]:
data_pivot_children = data.pivot_table(index=['children'], values='debt', aggfunc='mean')
data_pivot_children

,debt
children,
0,0.076541
1,0.093454
2,0.095508
3,0.082067
4,0.100000
5,0.000000


In [54]:
data_pivot_children = data.pivot_table(index=['children_group'], values='debt', aggfunc='mean')
data_pivot_children

,debt
children_group,
Есть дети,0.093457
Нет детей,0.076541


**Вывод**

<div class="alert alert-info">
<b> Видно, что люди с 5 детьми 100% возвращают вредит в срок. В целом можно сказать, отмечается резкое уменьшение процента невыплачиваемости у людей с 4 и 5 детьми. Думаю, что люди, имеющие столько детей, шли на такой шаг, как кредит максимально осознанно, взвешивая все риски, поэтому среди них столь малый процент. 
Самый высокий процент виден среди людей с 1 и 2 детьми (9,3% и 9,5%). Что логично, ведь чаще это те самые "молодые семьи", которые только становятся, чьи потребности растут быстрее, чем их заработок. Такие люди чаще рассматривают варианты взятия кредита, но вот не всегда могут возвращать кредит в срок.
    
    
9,3 % заемщиков с детьми - имеют задолжность по возврату кредита. В то время как среди заемщиков без детей, этот процент меньше - 7,6 %. Возможно это связано с тем, что люди, имеющие детей имеют и дополнительные траты на детей (часто могут быть непредвиденные траты), и из-за чаще не возвращают кредит в срок.


</b>
<br>
</div>

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Расчитываю долю неплатильщиков среди всех категорий семейного положения

In [55]:
data_pivot_family = data.pivot_table(index=['family_status'], values='debt', aggfunc='mean')
data_pivot_family

,debt
family_status,
Не женат / не замужем,0.098420
в разводе,0.071249
вдовец / вдова,0.066808
гражданский брак,0.094083
женат / замужем,0.076936


**Вывод**

<div class="alert alert-info">
<b> 9,8 % заемщиков, не женатых/не замужних - имеют задолжность по возврату кредита. Примерно такой же процент (9,4%) имеют люди, состоящие в гражданском браке. Возможно это связано с тем, что люди, не состоящие в официальном браке не чувствуют официальных обязательств перед второй половинкой и такие люди более склонны к риску, или они одиночки и их доход объективно меньше, чем у семейного человека (особенно если оба супруга работают). 
    
    
Интересно, что люди в разводе имеют примерно одинаковый, но все же меньший процент задолжности по возврату кредита (7,1 %), чем женатые/замужние люди (7,6 %). Считаю, что люди в разводе уже "почувстовали" настоящую жизнь 😀 прошли многие трудности и стараются планировать траты. 
    
Самый низкий процент невозвращаемости кредита в срок имеет категория "вдовец/вдова". Думаю, потому что такие люди чаще всего уже состоявшиеся личности, имеющие стабильную работу и доход, умеющие планировать свои траты.  
</b>
<br>
</div>

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Расчитываю долю неплатильщиков среди 4 категорий, разделенных по ежемесячному доходу

In [56]:
data_pivot_income = data.pivot_table(index=['total_income_group'], values='debt', aggfunc='mean')
data_pivot_income

,debt
total_income_group,
"(20666.999, 106808.0]",0.078788
"(106808.0, 142594.0]",0.093182
"(142594.0, 196975.5]",0.086191
"(196975.5, 2265604.0]",0.071212


**Вывод**

<div class="alert alert-info">
<b> 9,3 % заемщиков, имеющих ежемесячный доход от 106808 до 142594, не возвращают кредит в срок. 
    
8,6 % заемщиков с ежемесячным доходом от 142594 до 196975, не возвращают кредит в срок. 
    
Видно, что все это люди, имеющие средний доход. Могу предположить, что это связано с тем, что такие люди плохо оценивают свою платежеспособность и склонны рисковать, производить необдуманные траты. Но так как не все из них имеют стабильную "финансовую подушку", их ежемесчный доход является их единственным активом, то нетрудно предположить, что среди людей со средним заработком обнаружился больший процент по невозврату кредита в срок.
    
Самый низкий процент имеет категория граждан с самым высоким ежемесячным доходом (от 196975 до 2265604) - 7,1 %. Может они и склонны рисоквать, но чаще всего у таких людей всегда есть стабильная и крепкая "финансовая подушка" (люди с высоким доходом как правило имеют пассивыный доход) и объективно высокий заработок, способный закрывать не только базовые нужды.
    
Так же низкий процент невозврата кредита в срок оказался у людей с низким уровнем ежемесячного дохода (от 20666 до 106808) - 7,8 %. Считаю, что такие люди более трезво оценивают свою платежеспособность и риски. Склонны заблаговременно обдумывать и планировать свои траты.   

</b>
<br>
</div>

- Как разные цели кредита влияют на его возврат в срок?

Расчитываю долю неплатильщиков среди 4 категорий целей кредита

In [57]:
data_pivot_purpose = data.pivot_table(index=['purpose'], values='debt', aggfunc='mean')
data_pivot_purpose

,debt
purpose,
автомобиль,0.094123
недвижимость,0.073832
образование,0.093340
свадьба,0.080659


**Вывод**

<div class="alert alert-info">
<b> Примерно одинаковый процент невозвращаемости кредита в срок имеют люди с целью "автомобиль" и "образование" - 9,4% и 9,3% соответственно. Думаю, для людей, выбравших целью кредита "образование", это связанно с тем, что некоторые не получают что образование, на которое взят кредит (нетдоводят цель до конца), либо такие люди в силу того, что тратят основную часть времени на само обучение, не могут работать на стабильной работе с четким графиком, чаще рассматривают вариант неполного рабочего дня, неофициальной работы, "колымов" (разового зароботка) и соответсвенно, не всегда имеют регулярный заработок, что влияет на погошение кредита в срок. 
Люди, выбравшие целью кредита "автомобиль" не всегда могут не выплачивать в срок, потому что расходы на содержание автомобиля оказались больше, чем они могли себе предпологать, поэтому такая категория людей может иметь просрочку по погашению кредита.
Люди, выбравшие целью кредита "свадьба" могут не выплачивать в срок, потому что поддаются эмоциям, спешно берут кредит, не до конца взвесив риски и финансовую нагрузку. К тому же, если взглянуть на статистику разводов по России, многие пары разводятся на первых годах совместной жизни, что безусловно влияет на выплачивание кредита в срок. Процент таких людей в данной категории - 8%.
Самый меньший процент невыплачивания кредита в срок имеет категория людей, выбравших целью "недвижимость" - 7,3 %, думаю, это связано с тем, что кредит выдается на большую сумму, большинство людей здраво оценивают свои риски и морально готовы к такому шагу, продумывая свои ежемесячные траты. Тем более, для большинства людей жилье является приоритетным среди других расходов, людям нужно где-то жить.
</b>
<br>
</div>

## Шаг 4. Общий вывод

<div class="alert alert-info">
<b> Получив входные данные от банка — статистика о платёжеспособности клиентов, я занялась их предобработкой: обнаружила и заполнила пропуски, выявила и устранила явные и неявные дубликаты. По ходу исследовани я обнаруживала ошибки в данных, успешно их устраняла. Для удобства я заменила тип данных с вещественного на целоцисленный, лемматизировала данные. В процессе я вновь проверяла на наличие дубликатов, при их выявлении - ликвидировала их. Главной причиной появления дубликатов считаю человеческий фактор, данные вводились невнимательно, повторяясь, с ошибками и с использованием разного регистра. Успено категоризировала количественные и неколичественные данные.
    
Все это помогло мне дать ответы на 4 главных вопроса заказчика — кредитного отдела банка. Я выяснила, как влияет семейное положение и количество детей клиента на факт погашения кредита в срок.
    
По данным, люди с 5 детьми 100% возвращают кредит в срок. В целом можно сказать, отмечается резкое уменьшение процента невыплачиваемости у людей с большим количеством детей.
Отмечена следущая закономерность: чем большее количество детей имеет человек, тем более осознанно он становится заемщиком, тем серьезней он подходит к тому, чтобы выплачивать кредит в срок. 
    
Самый ненадежной категорией людей, имеющих высокий процент невыплачивания кредита в срок, оказались люди с 1 и 2 детьми (9,3% и 9,5%). Что логично, ведь чаще это молодые семьи, они еще не умеют четко планировать свой бюджет, ежемесячные траты, но могут остро нуждаться в решении многих житейских вопросов (покупка недвижимости для пополнишейся семьи, приобретение автомобиля, а то и все сразу).
В целом, если разделять заемщиков на людей с детьми и без, то более надежны люди, не имеющие детей (9,3% против 7,6%).
    
 
Мне показалось занимательным, та закономерность, которую я обнаружила, изучая влияние семейного положения на возврат кредита в срок:    
Выявила, что 9,8 % заемщиков, не женатых/не замужних - имеют задолжность по возврату кредита. Примерно такой же процент (9,4%) имеют люди, состоящие в гражданском браке. Возможно это связано с тем, что люди, не состоящие в официальном браке не чувствуют официальных обязательств перед второй половинкой и такие люди более склонны к риску, или они одиночки и их доход объективно меньше, чем у семейного человека (особенно если оба супруга работают).
Интересно, что люди в разводе имеют примерно одинаковый, но все же меньший процент задолжности по возврату кредита (7,1 %), чем женатые/замужние люди (7,6 %). Считаю, что люди в разводе уже прошли многие трудности и стараются планировать траты.

Самой надежной категорией с низким процентом невозвращаемости кредита в срок - категория "вдовец/вдова". Думаю, потому что такие люди чаще всего уже состоявшиеся личности, имеющие стабильную работу и доход, умеющие планировать свои траты.

Также я выяснила интересную особенность, связанную с уровнем ежемесячного дохода заемщиков:   
9,3 % заемщиков, имеющих ежемесячный доход от 106808 до 142594, не возвращают кредит в срок.
8,6 % заемщиков с ежемесячным доходом от 142594 до 196975, не возвращают кредит в срок. 
И эти люди со средним уровнем ежемесячного дохода! Могу предположить, что это связано с тем, что такие люди плохо оценивают свою платежеспособность и склонны рисковать, производить необдуманные траты. Но так как не все из них имеют стабильную "финансовую подушку", их ежемесчный доход является их единственным активом, то нетрудно предположить, что среди людей со средним заработком обнаружился больший процент по невозврату кредита в срок.

Самый надежная категория с низким процентом непогощаемости кредита в срок - категория граждан с самым высоким ежемесячным доходом (от 196975 до 2265604) - 7,1 %. Может они и склонны рисоквать, но чаще всего у таких людей всегда есть стабильная и крепкая "финансовая подушка" (люди с высоким доходом как правило имеют пассивыный доход) и объективно высокий заработок, способный закрывать не только базовые нужды.

Так же надежной оказалась категория людей с низким уровнем ежемесячного дохода (от 20666 до 106808) - их процент неуплаты кредита в срок - 7,8 %. Считаю, что такие люди более трезво оценивают свою платежеспособность и не готовы на какие-либо риски. Склонны заблаговременно обдумывать и планировать свои ежемесячные траты.



Я пришла к очень логичным выводам, исследуя зависимость цели кредита и влияния на его возврат в срок:    
Примерно одинаковый процент невозвращаемости кредита в срок имеют люди с целью "автомобиль" и "образование" - 9,4% и 9,3% соответственно. Думаю, для людей, выбравших целью кредита "образование", это связанно с тем, что некоторые не получают что образование, на которое взят кредит (нетдоводят цель до конца), либо такие люди в силу того, что тратят основную часть времени на само обучение, не могут работать на стабильной работе с четким графиком, чаще рассматривают вариант неполного рабочего дня, неофициальной работы, "колымов" (разового зароботка) и соответсвенно, не всегда имеют регулярный заработок, что влияет на погошение кредита в срок. Люди, выбравшие целью кредита "автомобиль" не всегда могут не выплачивать в срок, потому что расходы на содержание автомобиля оказались больше, чем они могли себе предпологать, поэтому такая категория людей может иметь просрочку по погашению кредита. Люди, выбравшие целью кредита "свадьба" могут не выплачивать в срок, потому что поддаются эмоциям, спешно берут кредит, не до конца взвесив риски и финансовую нагрузку. К тому же, если взглянуть на статистику разводов по России, многие пары разводятся на первых годах совместной жизни, что безусловно влияет на выплачивание кредита в срок. Процент таких людей в данной категории - 8%. Самый меньший процент невыплачивания кредита в срок имеет категория людей, выбравших целью "недвижимость" - 7,3 %, думаю, это связано с тем, что кредит выдается на большую сумму, большинство людей здраво оценивают свои риски и морально готовы к такому шагу, продумывая свои ежемесячные траты. Тем более, для большинства людей жилье является приоритетным среди других расходов, людям нужно где-то жить.

</b>
<br>
</div>